# Regression

## Cooks Distance

#### Cook’s distance is a measurement that uses both an observation’s leverage and its residual value. As the leverage is higher, its calculated Cook’s distance is higher. In general, a Cook’s distance value greater than 0.5 means a residual has leverage that is negatively impacting the model through over-representation – essentially, an outlier. Cook’s distance is included in the code and flags the residuals that have distances greater than 0.5. Cook’s distance is particularly useful because removing outliers may not have a significant impact on the model. However, if the outlier has a high Cook’s distance value, that is a strong indication that resolving the outlier will benefit the model. Cook’s distance is calculated for each data point by removing it from the model and calculating the difference in error divided by the mean squared error multiplied by the number of model coefficients plus one. 

## Leverage

#### Leverage is used to identify the distance of an individual point from all other points. High leverage for a residual likely means the corresponding data point is an outlier strongly influencing the model to fit less approximately to the overall data and instead give more weight to that specific value. Residuals should be between -2 and 2 to not be considered potential outliers. Values between +/-2.5 to 3 suggest data points are extreme outliers. Overall, this plot is useful for separating outliers that have no significant negative impact on the model from the ones that do.


## Handling serial correlation

#### If uncertain of the presence of serial correlation in the residuals, a useful next step is to analyze a Partial Autocorrelation Function (PACF) plot to assess whether serial correlation exists in the model at a significant level, which could explain issues with the model’s residuals.